<br>
<a href="https://www.nvidia.cn/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用大语言模型（LLM）构建 AI 智能体</h1>
<h2><b>评估：</b> 创建一个基础的研究智能体</h2>
<br>

**欢迎参加评估！** 我们希望您准备好运用到目前为止所学的一些技能，去构建一个您可能已经看到的东西；“研究型”聊天机器人。整体想法大家现在应该比较熟悉了：

- **聊天机器人应该能够理解您的问题，并在网络上查找一些资源。**
- **根据这些资源，聊天机器人应该能够根据其获取的信息做出合理的猜测。**

这通常与像 ChatGPT 和 Perplexity 这样的 LLM 接口结合使用，并且各种开源努力也应运而生，以简化这个过程。话虽如此，它们通常不依赖于 8B 模型，因为路由的复杂性。因此，我们将仅测试您实现以下基本功能的能力：
- **一个结构化输出接口，用于生成可解析的列表。**
- **一个用于搜索网页片段并筛选出最相关结果的函数。**
- **一个收集用户控制之外的消息的机制。**
- **一些基本的提示工程工具。**

值得注意的是，您现在应该能够想象出许多扩展。也许我们可以在某处加入重新查询的机制？或者用户或智能体可以批评并从历史记录中移除条目？毕竟，长期记忆听起来确实很吸引人。不过，我们将专注于实现这些简单功能，主要出于两个原因：
- **首先，我们真的不想强迫您做过多的工程设计。** 像 LangGraph 这样的框架可能做了很多努力，试图通过引入新的基本功能来简化接口，因此我们现在的任何过度设计，等您再看的时候可能会变得不再适用，因为会有一些更简单的现成选择。
- **其次，我们的 Llama-3.1-8B 模型因其限制而使得这个过程更加具挑战性。** 理解并处理这一挑战是很重要的，因为您将更好地分解更难的挑战，并在规模化时充分利用您的工具。话虽如此，当前用 Llama-8B 实现的多轮长期记忆研究智能体确实很繁琐，许多 streamlined 接口都假设了更强的模型。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

<hr><br>

## **第一部分：** 定义规划器

对于初始系统，请创建一个最小可行的“监督者”风格（'supervisor'-style）的元素，它尝试委派任务。这是一个非常模糊的定义，因此从技术上讲，生成任务列表的模块理论上是可行的。让我们从这个开始吧！

In [ ]:
from pydantic import BaseModel, Field
from functools import partial
from typing import List

from course_utils import SCHEMA_HINT

##################################################################
## TODO: Create an LLM client with the sole intention of generating a plan.

class Plan(BaseModel):
    ## TODO: Define a variable of choice, including useful prompt engineering/restrictions
    pass

planning_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a master planner system who charts out a plan for how to solve a problem."
        ## TODO: Perform some more prompt engineering. Maybe consider including the schema_hint
    )),
    ("placeholder", "{messages}"),
])

## TODO: Construct the necessary components to create the chain
planning_chain = None

input_msgs = {"messages": [("user", "Can you help me learn more about LangGraph?")]}

## For convenience, we have defined a 
step_buffer = []
for chunk in planning_chain.stream(input_msgs):
    if "steps" in chunk:
        if len(chunk.get("steps")) > len(step_buffer):
            if step_buffer:
                print(flush=True)
            step_buffer += [""]
            print(" - ", end='', flush=True)
        dlen = len(chunk.get("steps")[-1]) - len(step_buffer[-1])
        step_buffer[-1] = chunk.get("steps")[-1]
        print(step_buffer[-1][-dlen:], end="", flush=True)

<br>

为了帮助将来模块化这个过程，您可以使用这个生成器包装器。过程基本相同，但会将结果输出以供调用者处理：

In [ ]:
def generate_thoughts(input_msgs, config=None):
    step_buffer = [""]
    for chunk in planning_chain.stream(input_msgs, config=config):
        if "steps" in chunk and chunk.get("steps"):
            if len(chunk.get("steps")) > len(step_buffer):
                yield step_buffer[-1]
                step_buffer += [""]
            dlen = len(chunk.get("steps")[-1]) - len(step_buffer[-1])
            step_buffer[-1] = chunk.get("steps")[-1]
    yield step_buffer[-1]
    print("FINISHED", flush=True)

from time import sleep

for thought in generate_thoughts(input_msgs):
    
    print("-", thought)
    
    ## Example Use-Case: Slowing down the generation
    # for token in thought:
    #     print(token, end="", flush=True)
    #     sleep(0.02)
    # print(flush=True)
    # sleep(0.1)

<hr><br>

## **任务 2：** 定义检索子过程机制

现在我们有了一系列想要考虑的步骤，咱们可以以此为基础来搜索互联网。尝试实现一个您选择的搜索机制，并尽可能并行化/批处理这个过程。

- 随意实现 `search_internet` 和 `retrieve_via_query`，与热身时的一致（`DuckDuckGoSearchAPIWrapper` + `NVIDIARerank`），或者可以写一个您觉得有趣的方案。实现一个循环（智能体作为工具？）可能会很有趣，在这里您可以搜索、扩展上下文、过滤，然后再搜索。概念上很简单，但实现起来要更复杂。
- 如果您想的话，可以使用 `tools` 格式，但这不是必需的。按照您觉得有趣的方式来做。
- 我们的解决方案确实在某一时刻使用了 `RunnableLambda(...).batch`。也可以尝试利用 `RunnableParallel`。这两个都可能有用，但并非必需。

In [ ]:
# from langchain_core.runnables import RunnableLambda
# from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
# import functools

####################################################################
## TODO: Implement a "step researcher" mechanism of choice
## We incorporated a 2-step process similar to the example notebook.

# @functools.cache  # <- useful for caching duplicate results
# def search_internet(final_query: str): 
#     ## OPTIONAL: We ended up defining this method
#     pass 
     
def research_options(steps):
    return [] ## TODO

search_retrievals = research_options(step_buffer)
# search_retrievals

In [ ]:
# from langchain_nvidia import NVIDIARerank
# from langchain_core.documents import Document

## Optional Scaffold
def retrieve_via_query(context_rets, query: str, k=5):
    return [] ## TODO

filtered_results = [retrieve_via_query(search_retrievals, step) for step in step_buffer]
# filtered_results

<hr><br>

## **第 3 部分：** 创建研究工作流

现在我们有了一些最低可行的监督/下属系统的雏形，咱们可以以有趣的方式来协调它们。随意想出您自己的“推理”问题和“研究”结果的机制。如果您找不到简单的解决办法，下面有一个默认的提示词池（可能是我们用过的）。

In [ ]:
## TODO: Define the structured prompt template. Doesn't have to be this!
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an agent. Please help the user out! Questions will be paired with relevant context."
     " At the end, output the most relevant sources for your outputs, being specific."
    ),
    ("placeholder", "{messages}"),
])

intermediate_prompt = "I can help you look into it. Here's the retrieval: {action} -> {result}" 
final_question = "Great! Now use this information to solve the original question: {question}"

In [ ]:
question = "Can you help me learn more about LangGraph?"
# question = "Can you help me learn more about LangGraph? Specifically, can you tell me about Memory Management?"
# question = "Can you help me learn more about LangGraph? Specifically, can you tell me about Pregel?"
# question = "Can you help me learn more about LangGraph? Specifically, can you tell me about subgraphs?"
# question = "Can you help me learn more about LangGraph? Specifically, can you tell me about full-duplex communication?"
# question = "Can you help me learn more about LangGraph? Specifically, can you tell me about productionalization?"
## TODO: Try your own highly-specialized questions that shouldn't be answerable from priors alone. 

input_msgs = {"messages": [("user", question)]}

#########################################################################
## TODO: Organize a systen  to reason about your question progressively.
## Feel free to use LangChain or LangGraph. Make sure to wind up with 
## a mechanism that that remembers the reasoning steps for your system

sequence_of_actions = [thought for thought in generate_thoughts(input_msgs)]
## ...

## HINT: We ended up with a for-loop that accumulated intermediate "question-answer" pairs
## You may also consider a map-reduce-style approach to operate on each step independently.

# for action, result in zip(sequence_of_actions, filtered_results):  ## <- possible start-point
#     pass

input_msgs["messages"] += []

# ## HINT: If you wind up with a chain, this may be easy to work with...
# print("*"*64)
# for token in chain.stream(input_msgs):
#     if "\n" in token:
#         print(flush=True)
#     else: 
#         print(token, end="", flush=True)

<hr><br>

## **第 4 部分：** 汇总您的推理路径（Reasoning Traces）

根据您的实现方式，最后的要求可能简单也可能需要稍微多花点力气。请汇总8个多样且合理的问题的答案，同时也收集推理路径（即“推理”，要呈现成可理解的格式）。

这个输出将由 LLM 进行评估，以检查响应是否看起来展现出了合理的行为（推理合理，最终输出解答了问题，引用了来源等等）。

In [ ]:
## TODO: Aggregate 8 question-trace-answer triples. 
# [ 
#   {"question": str, "trace": list or dict or str, "answer": str}, 
#   ...
# ]

submission = [{}]

<hr>
<br>

## **第 5 部分：** 运行评估

为了评估您的提交，请运行以下单元以保存您的结果，接下来运行以查询评估运行器的单元。

**遵循指示，确保一切都能通过。**

In [ ]:
import requests

## Send the submission over to the assessment runner
response = requests.post(
    "http://docker_router:8070/run_assessment", 
    json={"submission": submission},
)

response.raise_for_status()

try: 
    print(response.json().get("result"))
    if response.json().get("messages"):
        print("MESSAGES:", "\n  - ".join([""] + response.json().get("messages")))
    if response.json().get("exceptions"):
        print("EXCEPTIONS:", "\n[!] ".join([""] + [str(v) for v in response.json().get("exceptions")]))
except:
    print("Failed To Process Assessment Response")
    print(response.__dict__)

<br>

如果您通过了评估，请返回到课程页面（如下所示）并点击 **"ASSESS TASK"** 按钮，这样就能生成您的课程证书了。

<img src="./images/assess_task.png" style="width: 800px;">

<hr>
<br>

## **第 6 部分：** 总结

### <font color="#76b900">**恭喜您完成课程！！**</font>

在结束本课程之前，我们强烈建议您下载课程资料以作日后参考，并查看课程的**"下一步"**和**反馈**部分。 **非常感谢您花时间完成这门课程，期待在系列课程中再次见到您！**